<a href="https://colab.research.google.com/github/laura314159265/poly/blob/main/SIMULACI%C3%93_A/MORO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt

# Dades en unitats del SI
G= 6.67430e-11
m_sol= 1.98847e30
m_terra= 5.97219e24
m_lluna= 7.349e22
m_mart= 6.4171e23

# Condicions inicials (en cartesianes)
# L'origen és el baricentre del sistema solar
# Format: [x,y,z,vx,vy,vz]
# Inici a les 13:59 del 21 de Juny del 2025 (solstici d'estiu)
# Això és per el temps central Europeu, per tant hi ha un lleuger error
# Ja que no està a Bellaterra/Barcelona exactament
# Posicions en kilometres i velocitats en km/s

lluna= np.array([-1.223926839954494E+05, -1.526112945830738E+08, 5.512426765490323E+04, 2.865929048320844E+01, 7.673984990113614E-01, 6.367954754048094E-02])
terra= np.array([-4.133776082794621E+05, -1.528302631740358E+08, 3.242787546376139E+04, 2.932642969193051E+01, -7.378251008835289E-02, -1.172300942425906E-03])
sol= np.array([-6.709778526787314E+05, -8.035450223548296E+05, 2.366174814126350E+04, 1.263745636318025E-02, -3.212119035252178E-03, -2.265926685896345E-04])
mart= np.array([-2.457240245390437E+08, -2.253913894859833E+07, 5.577461384361338E+06, 3.057555929366051E+00, -2.206677199548648E+01, -5.372712135532627E-01])

# Normalització
t0= 86400.0            # Temps característic= 1 dia
m0= m_sol              # Massa característica= massa del sol
r0= (G * t0**2 * m0)**(1/3)  # Radi característic

def normalitza(cos):
  pos = cos[:3]*(1/r0)*10**3
  vel = cos[3:]*(t0/r0)*10**3
  return pos, vel

masses= np.array([m_sol/m0, m_terra/m0, m_lluna/m0, m_mart/m0])
pos_sol, vel_sol= normalitza(sol)
pos_terra, vel_terra= normalitza(terra)
pos_lluna, vel_lluna= normalitza(lluna)
pos_mart, vel_mart= normalitza(mart)
posicions= np.array([pos_sol, pos_terra, pos_lluna, pos_mart])
velocitats= np.array([vel_sol, vel_terra, vel_lluna, vel_mart])

def acceleracions(pos, masses): # pos és una matriu de N files i 3 columnes
    N= len(masses) # Nombre de cossos: planetes, el Sol i la Lluna
    acc= np.zeros_like(pos) # Crea un array amb la mateixa forma que pos però ple de zeros
    for i in range(N):
        for j in range(N):
            if i != j:  # Si i ≠ j, perquè un cos no s'atrau a ell mateix
                diff= pos[j] - pos[i]
                d= np.linalg.norm(diff) # Calcula la norma del vector diferència de posicions
                acc[i]= acc[i] + masses[j] * diff / d**3
    return acc

def rk4(pos, vel, masses, h):
    K1= vel
    L1= acceleracions(pos, masses)
    K2= vel + 0.5*h*L1
    L2= acceleracions(pos + 0.5*h*K1, masses)
    K3= vel + 0.5*h*L2
    L3= acceleracions(pos + 0.5*h*K2, masses)
    K4= vel + h*L3
    L4= acceleracions(pos + h*K3, masses)
    pos_nou= pos + (h/6)*(K1 + 2*K2 + 2*K3 + K4)
    vel_nou= vel + (h/6)*(L1 + 2*L2 + 2*L3 + L4)
    return pos_nou, vel_nou

t_f = 365
dt = 1/24
hores= int(t_f/dt)
passos = hores +6  #per tenir en compte que un any dura 365 dies i 6 hores afegim 5 + CI = 6 hores "extres" als 365 dies calculats
#AIXÒ ENS DONA L'HORA JUST ABANS DE RETORNAR A LES CONDICIONS INICIALS. SI VOLEM TROBAR EL MOMENT DEL QUE HEM PARTIT PERO UN ANY DESPRÉS HEM DE POSAR EL TEMPS SEGÜENT ÉS A DIR UN + 6

pos_hist = np.zeros((passos+1, len(masses), 3), dtype=float)
vel_hist = np.zeros((passos+1, len(masses), 3), dtype=float)
pos_hist[0] = posicions
vel_hist[0] = velocitats

for i in range(passos):
  pos_nou, vel_nou = rk4(pos_hist[i], vel_hist[i], masses, dt)
  pos_hist[i+1] = pos_nou
  vel_hist[i+1] = vel_nou

# Extreure trajectòries
idx_sol = 0
idx_terra = 1

x_t = pos_hist[:, idx_terra, 0]
y_t = pos_hist[:, idx_terra, 1]
z_t = pos_hist[:, idx_terra, 2]

x_s = pos_hist[:, idx_sol, 0]
y_s = pos_hist[:, idx_sol, 1]
z_s = pos_hist[:, idx_sol, 2]

# Volem el vector que uneix sol i terra llavors farem el vector baricentre-Terra menys el vector baricentre-Sol

R_t = 6371000 #metres
vel_ang = 2*np.pi/23.9344444 # en rad/h
theta = (23.43333333333333*2*np.pi)/360
phi= phi= (41.505833333*2*np.pi)/360


t = np.linspace(1,passos-1,passos)

# Aquestes són les coordenades del vector centre terra-bellaterra durant el temps que considerem més condicions inicials
def rodrigues (alpha):
  x = np.array([0])
  y = np.array([R_t*np.sin(alpha + np.pi/2 - (phi-theta))])
  z = np.array([R_t*np.cos(alpha + np.pi/2 - (phi-theta))])
  X = np.concatenate((x,(R_t * np.sin(vel_ang * t))*(-np.cos(theta)*np.sin(alpha + np.pi/2 - (phi-theta)) + np.sin(theta)*np.cos(alpha + np.pi/2 - (phi-theta)))))
  Y = np.concatenate((y,(np.sin(alpha + np.pi/2 - (phi-theta))*np.cos(vel_ang*t) + np.sin(theta)*(np.sin(theta)*np.sin(alpha + np.pi/2 - (phi-theta)) + np.cos(theta)*np.cos(alpha + np.pi/2 - (phi-theta)))*(1 - np.cos(vel_ang*t))) * R_t))
  Z = np.concatenate((z,(np.cos(alpha + np.pi/2 - (phi-theta))*np.cos(vel_ang*t) + np.cos(theta)*(np.sin(theta)*np.sin(alpha + np.pi/2 - (phi-theta)) + np.cos(theta)*np.cos(alpha + np.pi/2 - (phi-theta)))*(1 - np.cos(vel_ang*t))) * R_t))
  return X, Y, Z

alphes =np.linspace(0,50*2*np.pi/360,51)

x_st = r0 * (x_t - x_s)
y_st = r0 * (y_t - y_s)
z_st = r0 * (z_t - z_s)
sum_cos =[]
X_b = 0
Y_b = 0
Z_b = 0
cos_sum=0

for alpha in alphes:
  res= rodrigues(alpha)
  X_b = res[0]
  Y_b = res[1]
  Z_b = res[2]

  mod_t = np.sqrt(x_st**2 + y_st**2 + z_st**2)
  Mod_b = np.sqrt(X_b**2 + Y_b**2 + Z_b**2)

  cos_llista = (X_b * x_st + Y_b * y_st + Z_b * z_st)/(Mod_b * mod_t)
  theta_llista = np.pi - np.arccos(cos_llista)

  cos_llista = np.cos(theta_llista)
  cos_mask = np.ma.masked_where(cos_llista < 0 , cos_llista)

  print(cos_mask)
  sum_cos.append(np.sum(cos_mask))
  cos_sum=list(sum_cos)
print(cos_sum)
print(cos_sum.index(np.max(cos_sum)))
print("AQUI",theta_llista[0] *360/(2*np.pi),theta_llista[-6]*360/(2*np.pi))



[0.9506454819896706 0.9275670059391399 0.8592576216880187 ...
 0.7494054642751447 0.6071392096979279 0.441467026212672]
[0.9558454999405085 0.9324189193713252 0.8630791695101182 ...
 0.7515700190151542 0.6071578180010904 0.4389866361633719]
[0.960762280240879 0.9369945369544522 0.8666449683475181 ...
 0.7535118673383202 0.6069965123392401 0.4363761648996693]
[0.9653943659426657 0.9412925028342818 0.8699539615770661 ...
 0.7552304338330761 0.6066553405107262 0.4336363859605375]
[0.9697403844589072 0.9453115434307654 0.873005168672655 ...
 0.7567252092510796 0.6061344036121245 0.4307681112016063]
[0.9737990479705215 0.9490504678154362 0.8757976854957714 ...
 0.7579957506581136 0.6054338560082807 0.4277721905545888]
[0.9775691538079176 0.9525081680643069 0.8783306845634132 ...
 0.7590416815653402 0.6045539052865688 0.42464951177542976]
[0.98104958480737 0.9556836195861704 0.8806034152932902 ...
 0.759862692040858 0.6034948121953787 0.42140100018124466]
[0.9842393096420597 0.95857588142620